# Data Engineer Certification Sample Practical Exam

HappyPaws, creates fun and educational apps for pet owners. 

HappyPaws wants to help pet owners understand their pets better by tracking their activities and health through the app. 

The data engineering team is responsible for making sure all the pet data from thousands of users is organized and safe, so pet owners can get tips to keep their pets happy and healthy.



## Task

HappyPaws has collected three datasets over the past year: 
 - "pet_activities.csv" which logs daily activities of pets, 
 - "pet_health.csv" which records vet visits and health issues, and 
 - "users.csv" which contains information about the pet owners. 

Each dataset contains unique identifiers for pets and/or their owners. 

The engineers developing the app currently write code to cross reference all of these data sources. 

They want to make things easier by having a single table with all data included.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset. 

The final dataset should provide a comprehensive view of each pet's activities, health records, and owner information. 

- To test your code, your manager will run only the code `all_pet_data('pet_activities.csv', 'pet_health.csv', 'users.csv')`
- Your `all_pet_data()` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.

## Data

The data that has been provided has the following structure

![image](image.png)

The function that you write must return data as described below. There should be a unique row for each activity/health visit. 

Where missing values are permitted, they should be in the default Python format.

|Column Name | Description |
|-------|-------|
| pet_id | Unique identifier for each pet. There should not be any missing values.|
| date | The date of the activity recorded or the date of the health visit, in date format. There should not be any missing values.|
| activity_type | The type of activity, one of 'Walking', 'Playing', 'Resting' or for rows that relate to a health visit, the value 'Health'. Missing values are permitted.|
| duration_minutes | The duration of the activity in minutes. For rows that relate to health visits, this should be 0. Missing values for other activities are permitted.|
| issue | The health issue identified or check-up note. For rows that relate to activities, this should be a missing value. Missing values for health activities are permitted.|
| resolution | The outcome or advice given for the issue. For rows that relate to activities, this should be a missing value. Missing values for health activities are permitted.|
| owner_id | Unique identifier for the pet owner. All pets must have an owner.|
| owner_age_group | The age group of the owner (e.g., 18-25, 26-35, etc.). Missing values are permitted.|
| pet_type | The type of pet (e.g., Dog, Cat). Missing values are permitted.|

## Code

### Imports

In [256]:
# Imports
import pandas as pd
import numpy as np

### Functions

In [257]:
# Load databases

def load_dbs(*filenames: str):
    """Takes supplied filenames and returns a list of dataframes \
    
    The list order is 'activities', 'health', 'users'

    Returns:
        list: a list of dataframes
    """
    names = ['activities', 'health', 'users']
    dataframes = []
    for name in names:
        result = list(filter(lambda filename: name in filename, filenames))
        if not result:
            raise ValueError(f'None of the supplied filenames contains "{name}"')
        if len(result) > 1:
            raise ValueError(f'More than one filename returned containing "{name}"')
        dataframes.append(pd.read_csv(result[0]))
    return dataframes

In [ ]:
# Merge databases

def merge_dbs(*dfs: pd.DataFrame, method='inner'):
    """Merges the supplied databases \
    
    Expects databases to be two or more of "pet activities", "pet health", and "users"

    Args:
        method (str, optional): The method to use for joining any database to "users". Defaults to 'inner'.

    Raises:
        ValueError: Raised if fewer than 2 valid dataframes are supplied

    Returns:
        pandas.DataFrame: the resulting merged dataframe
    """
    # Identify dataframes
    activities = list(filter(lambda x: 'activity_type' in x.columns, dfs))
    health = list(filter(lambda x: 'issue' in x.columns, dfs))
    users = list(filter(lambda x: 'owner_id' in x.columns, dfs))

    # Check for at least two valid databases
    if sum([bool(x) for x in [activities, health, users]]) < 2:
        raise ValueError('Must supply at least two valid dataframes')

    # Rename columns to simplify merge
    if health:
        health[0].rename(columns={'visit_date': 'date'}, inplace=True)

    # Merge dataframes
    if activities and health:
        activities = pd.concat([activities[0], health[0]], ignore_index=True)
    elif health:
        activities = health[0]
    else:
        activities = activities[0]
    if users:
        merged_df = pd.merge(activities, users[0], on='pet_id', how=method)
    else:
        merged_df = activities
    return merged_df

In [ ]:
# Clean data

def clean_data(df: pd.DataFrame):
    """Checks for specific columns in the dataframe and cleans them

    Args:
        df (pandas.DataFrame): the dataframe to be cleaned

    Returns:
        pandas.DataFrame: the cleaned dataframe
    """
    tdf = df.copy()

    # Drop records missing "pet_id", "owner_id", or "date"

    tdf.dropna(subset=['pet_id', 'date', 'owner_id'], inplace=True)

    # Strip leading and trailing whitespace from columns with strings
    for column in ['activity_type', 'issue', 'resolution', 'owner_age_group', 'pet_type']:
        if column in tdf.columns:
            tdf[column] = tdf[column].str.strip()
    
    # Change "date" datatype
    if 'date' in tdf.columns:
        tdf.date = pd.to_datetime(tdf.date, errors='coerce')

    # Assign "Health" to health activities
    if 'issue' in tdf.columns:
        tdf.loc[~df.issue.isna(), 'activity_type'] = 'Health'  

    # Clean values for 'activity_type': "Playing", "Walking", "Resting"
    if 'activity_type' in tdf.columns:
        for activity in ['Play', 'Walk', 'Rest']:
            tdf.activity_type = tdf.activity_type.str.replace(fr'^{activity}$', f'{activity}ing', regex=True)
        tdf.activity_type = tdf.activity_type.astype('category')  

    # Assign 0 to "duration_minutes" for health records and convert datatype
    if 'duration_minutes' in tdf.columns:
        tdf.loc[tdf.activity_type == 'Health', 'duration_minutes'] = 0
        tdf.loc[tdf.duration_minutes == '-', 'duration_minutes'] = np.nan
        tdf.duration_minutes = tdf.duration_minutes.astype('float')

    # Assign NaN for "issue", if missing, and change datatype to "category"
    if 'issue' in tdf.columns:
        tdf.loc[tdf.issue.isna(), 'issue'] = np.nan
        tdf.issue = tdf.issue.astype('category')

    # Assign NaN for "resolution", if missing, and change datatype to "category"
    if 'resolution' in tdf.columns:
        tdf.loc[tdf.resolution.isna(), 'resolution'] = np.nan
        tdf.resolution = tdf.resolution.astype('category')

    # Change "owner_age_group" datatype to "category" and assign NaN, if missing
    if 'owner_age_group' in tdf.columns:
        tdf.loc[tdf.owner_age_group.isna(), 'owner_age_group'] = np.nan
        # tdf.owner_age_group = tdf.owner_age_group.astype('category')

    # Change "pet_type" datatype to "category" and assign NaN, if missing
    if 'pet_type' in tdf.columns:
        tdf.loc[tdf.pet_type.isna(), 'pet_type'] = np.nan
        tdf.pet_type = tdf.pet_type.astype('category')

    return tdf

In [ ]:
# Sort data
def sort_data(df: pd.DataFrame):
    """Orders the columns of the data and sorts the records
     Records are sorted by `pet_id`, `date`, `owner_id`, and `activity_type`

    Args:
        df (pd.DataFrame): the dataframe to be sorted

    Returns:
        pandas.DataFrame: a dataframe that has been ordered
    """
    col_order = ['pet_id', 'date', 'activity_type', 'duration_minutes', 'issue', 'resolution',	'owner_id',	'owner_age_group', 'pet_type']
    cols = [x for x in col_order if x in df.columns]
    sorted_df = df.loc[:, cols].sort_values(['pet_id', 'date', 'owner_id', 'activity_type']).reset_index(drop=True)
    return sorted_df

In [ ]:
# Get pet data (final function)

def all_pet_data(activities: str, health: str, users: str, merge: str = 'left', sort: bool = False):
    """Merges and cleans data from the "pet activities", "pet health", and "users" databases

    Args:
        activities (str): the "activities" CSV file
        health (str): the "health" CSV file
        users (str): the "users" CSV file
        merge (str): the type of merge to be performed between any database and "users". Defaults to 'left'.
        sort (bool): whether to sort rows and columns of the resulting dataframe. Defaults to False.

    Returns:
        pandas.DataFrame: a DataFrame of cleaned and merged databases
    """
    df_list = load_dbs(activities, health, users)   # Load
    merge_df = merge_dbs(*df_list, method=merge)    # Merge
    clean_df = clean_data(merge_df)                 # Clean
    if sort:                                        # Sort
        return sort_data(clean_df)
    return clean_df

In [262]:
# Test function

def test_function(activities: str, health: str, users: str):

    with open (activities, 'r') as file:
        pa_df=pd.read_csv(file, parse_dates=['date'])

    with open (health, 'r') as file:
        ph_df=pd.read_csv(file, parse_dates=['visit_date'])

    with open (users, 'r') as file:
        u_df=pd.read_csv(file)

    pa_df = pa_df.dropna(subset=['pet_id', 'date'])
    pa_df['activity_type'] = pa_df['activity_type'].str.strip()
    pa_df['activity_type'] = pa_df['activity_type'].replace({"Play":"Playing", "Walk":"Walking", "Rest":"Resting"})
    pa_df['issue'] = pa_df['issue'] = np.nan
    pa_df['resolution'] = np.nan

    ph_df = ph_df.rename(columns={'visit_date':'date'})
    ph_df = ph_df.dropna(subset=['pet_id', 'date'])
    ph_df['issue'] = ph_df['issue'].str.strip()
    ph_df['resolution'] = ph_df['resolution'].str.strip()
    ph_df['activity_type'] = 'Health'
    ph_df['duration_minutes'] = 0

    u_df = u_df.dropna(subset=['owner_id'])
    u_df['owner_age_group'] = u_df['owner_age_group'].str.strip()
    u_df['pet_type'] = u_df['pet_type'].str.strip()

    pa_ph_df = pd.concat([pa_df,ph_df], axis=0, ignore_index=True)

    final_df = pa_ph_df.merge(u_df, on='pet_id', how='left')

    final_df['duration_minutes'] = final_df['duration_minutes'].replace('-', np.nan)

    #final_df['activity_type'] = final_df['activity_type'].astype('category')
    final_df['duration_minutes'] = final_df['duration_minutes'].astype(float)
    #final_df['issue'] = final_df['issue'].astype('category')
    #final_df['resolution'] = final_df['resolution'].astype('category')
    #final_df['owner_age_group'] = final_df['owner_age_group'].astype('category')
    #final_df['pet_type'] = final_df['pet_type'].astype('category')

    #final_df.info()

    return final_df

In [263]:
filelist = ['pet_activities.csv', 'pet_health.csv', 'users.csv']
df_left = all_pet_data(*filelist)
df_inner = all_pet_data(filelist[0], filelist[1], filelist[2], 'inner')
test_df = test_function(*filelist)

from pandas.testing import assert_frame_equal
assert_frame_equal(df_left, df_inner)
assert_frame_equal(test_df, df_left)

AssertionError: DataFrame.iloc[:, 0] (column name="pet_id") are different

DataFrame.iloc[:, 0] (column name="pet_id") values are different (99.73376 %)
[index]: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]
[left]:  [9930, 3040, 2381, 3936, 8989, 627, 3132, 9988, 3735, 3035, 4339, 3686, 9230, 3223, 3612, 7393, 1001, 7267, 6725, 3225, 4238, 1956, 2058, 6332, 9689, 9419, 2178, 3001, 9689, 9769, 5468, 2549, 7956, 5747, 1382, 2869, 5975, 8805, 8244, 8007, 6103, 2882, 1297, 850, 1640, 5071, 4583, 5062, 9305, 1585, 8285, 8989, 7818, 427, 9223, 1925, 7030, 9367, 3088, 2342, 9539, 4150, 4001, 1906, 585, 8226, 579, 7022, 3846, 850, 5984, 7716, 4480, 